In [ ]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_documents=loader.load()
text_documents

ModuleNotFoundError: No module named 'langchain_community'